# Maintenance simulation

A workshop works continuously with *n* machines. A stream of jobs arrives. Each machine breaks down
  periodically, repairs are carried out by one repairman (who hasother, less important tasks to perform, too). Broken machines
  are tested, then continue.

In [5]:
import random as rnd
import simpy 

In [6]:
seed = 42
proc_mean = 10.0         
proc_stdev= 2.0         
mean_to_failure = 300
failure_mean = 1 / mean_to_failure  
repair_time = 30    
task_time = 30    
nMachines = 10    
weeks = 4             
sim_time = weeks * 7 * 24 * 60  

In [10]:
def proc_time():
    return rnd.normalvariate(proc_mean, proc_stdev)

def time_to_failure():
    return random.expovariate(failure_mean)


class Machine():
    
    def __init__(self, env, name, repairman):
        self.env = env
        self.name = name
        self.repairman = repairman
        self.nparts = 0
        self.broken = False
        self.process = env.process(self.working(repairman))
        env.process(self.failure())

    def working(self, repairman):
        while True:
            ptime = proc_time()
            while ptime > 0:
                try:
                    start = self.env.now
                    yield self.env.timeout(ptime)
                    ptime = 0

                except simpy.Interrupt:
                    self.broken = True
                    ptime -= self.env.now - start  

                    with repairman.request(priority=1) as req:
                        yield req
                        yield self.env.timeout(repair_time)

                    self.broken = False

            self.nparts += 1

    def failure(self):
        while True:
            tfailure = time_to_failure()
            yield self.env.timeout(tfailure)
            if not self.broken:
                self.process.interrupt()

def low_priority_tasks(env, repairman):
    while True:
        ttime = task_time
        while ttime:
            with repairman.request(priority=2) as req:
                yield req
                try:
                    start = env.now
                    yield env.timeout(ttime)
                    ttime = 0
                except simpy.Interrupt:
                    ttime -= env.now - start

In [14]:
print('Machine shop \n')
random.seed(seed) 

env = simpy.Environment()
repairman = simpy.PreemptiveResource(env, capacity=1)
machines = [Machine(env, str(i), repairman) for i in range(nMachines)]
env.process(low_priority_tasks(env, repairman))

env.run(until=sim_time)

print('Machine shop results after', weeks, ' weeks')
for machine in machines:
    print(machine.name, ' made ', machine.nparts, ' parts.')

Machine shop 

Machine shop results after 4  weeks
0  made  3251  parts.
1  made  3273  parts.
2  made  3242  parts.
3  made  3343  parts.
4  made  3387  parts.
5  made  3244  parts.
6  made  3269  parts.
7  made  3185  parts.
8  made  3302  parts.
9  made  3279  parts.


### Credits & Links

https://simpy.readthedocs.io/en/latest/examples/machine_shop.html